In [1]:
import sys, os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from typing import List, Tuple, Callable, Any, Dict

from methods.PolyCG.polycg.transforms.transform_marginals import send_to_back_permutation
from methods.midstep_composites import midstep_composition_transformation, midstep_groundstate, midstep_groundstate_se3
from methods.midstep_composites import midstep_composition_transformation_correction, midstep_composition_transformation_correction_old
from methods.read_nuc_data import read_nucleosome_triads, GenStiffness
from methods.free_energy import calculate_midstep_triads, midstep_excess_vals
from methods.PolyCG.polycg.SO3 import so3

from methods.nearest_correlation import nearcorr

# from methods.PolyCG.polycg.SO3 import so3
# from methods.PolyCG.polycg.transforms.transform_SO3 import euler2rotmat_so3
# from methods.PolyCG.polycg.cgnaplus import cgnaplus_bps_params
# from methods.PolyCG.polycg.transforms.transform_algebra2group import algebra2group_lintrans, group2algebra_lintrans

np.set_printoptions(linewidth=250,precision=5,suppress=True)

-----
-----
# Methods
-----
-----

### Matrix Methods

In [2]:
def plot_matrix(mat,dims=6,cmap='gray_r'):
    mat = np.copy(mat)
    fig = plt.figure(figsize=(17./2.54*1.3,17.2/2.54))
    ax1 = fig.add_subplot(111)
    
    img = ax1.imshow(mat, interpolation='none', cmap=cmap)
    # plot blocks
    for l in range(1,len(mat)//dims+1):
        ax1.plot([l*dims-0.6,l*dims-0.6],[0-0.5,len(mat)-0.5],color='black',lw=0.5)
        ax1.plot([0-0.5,len(mat)-0.5],[l*dims-0.4,l*dims-0.4],color='black',lw=0.5)

        ax1.plot([l*dims-0.6,l*dims-0.6],[(l-1)*dims-0.5,l*dims-0.5],color='red',lw=0.5)
        ax1.plot([(l-1)*dims-0.6,(l-1)*dims-0.6],[(l-1)*dims-0.5,l*dims-0.5],color='red',lw=0.5)
        ax1.plot([(l-1)*dims-0.5,l*dims-0.5],[l*dims-0.4,l*dims-0.4],color='red',lw=0.5)
        ax1.plot([(l-1)*dims-0.5,l*dims-0.5],[(l-1)*dims-0.4,(l-1)*dims-0.4],color='red',lw=0.5)
        
    ax1.set_xlim((-0.5,len(mat)-.5))
    ax1.set_ylim((-0.5,len(mat)-.5))
    # ax1.set_xlim((-0.5,12-.5))
    # ax1.set_ylim((-0.5,12-.5))
    fig.colorbar(img, ax=ax1)
    plt.show()
    
def regularize_matrix(A, epsilon=1e-6):
    # (Optional) ensure A is symmetric to avoid floating-point asymmetries
    A_sym = 0.5 * (A + A.T)
    # Eigen-decomposition for symmetric (real) matrices
    eigenvals, Q = np.linalg.eigh(A_sym)
    # Lift eigenvalues below epsilon to epsilon
    eigenvals_clipped = np.clip(eigenvals, epsilon, None)
    # Reconstruct the matrix
    A_reg = Q @ np.diag(eigenvals_clipped) @ Q.T
    # A_reg is guaranteed to be real and symmetric
    A_reg = A_reg.astype(np.float64)
    return A_reg

def block_sum(M,dims):
    if len(M)%dims != 0:
        raise ValueError('Matrix dimension not a multiple of dims.')
    N = len(M)//dims
    bM = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            bM[i,j] = np.max(M[i*dims:(i+1)*dims,j*dims:(j+1)*dims])
    return bM

def is_positive_definite(matrix):
    if not np.allclose(matrix, matrix.T):
        print('Not allclose')
        return False
    eigenvalues = np.linalg.eigvals(matrix)
    return np.all(eigenvalues > 0)

def is_regular(A):
    if A.shape[0] != A.shape[1]:
        return False
    return np.linalg.matrix_rank(A) == A.shape[0]

def marginalize_diagonal_blocks(K: np.ndarray,dim: int=6):
    cov = np.linalg.inv(K)
    dcov = np.zeros(cov.shape)
    for i in range(len(K)//dim):
        dcov[i*dim:(i+1)*dim,i*dim:(i+1)*dim] = cov[i*dim:(i+1)*dim,i*dim:(i+1)*dim]
    dK = np.linalg.inv(dcov)
    return dK


### Selection Methods

In [3]:
def get_midstep_locations(left_open: int, right_open: int, base_locations = None, sort = True):
    if base_locations is None:
        MIDSTEP_LOCATIONS = [
            2, 6, 14, 17, 24, 29, 
            34, 38, 45, 49, 55, 59, 
            65, 69, 76, 80, 86, 90, 
            96, 100, 107, 111, 116, 121, 
            128, 131, 139, 143
        ]
    else:
        MIDSTEP_LOCATIONS = base_locations
    if left_open + right_open > len(MIDSTEP_LOCATIONS):
        return []
    
    locs = MIDSTEP_LOCATIONS[left_open:len(MIDSTEP_LOCATIONS)-right_open]
    if sort:
        locs = sorted(list(set(locs)))
    return locs

def submatrix(M, dim: int = 6, left_open: int = 0, right_open: int = 0, marginalize=False):
    n = len(M)//dim
    if marginalize:
        return np.linalg.inv(np.linalg.inv(M)[left_open*dim:(n-right_open)*dim,left_open*dim:(n-right_open)*dim])
    return M[left_open*dim:(n-right_open)*dim,left_open*dim:(n-right_open)*dim]

### Binding Model Methods

In [4]:
def Hinverse(Psi):
    psih = so3.hat_map(Psi)
    psihsq = psih @ psih
    Hinv = np.eye(3)
    Hinv += 0.5* psih
    Hinv += 1./12 * psihsq
    Hinv -= 1./720 * psihsq @ psihsq
    Hinv += 1./30240 * psihsq @ psihsq @ psihsq
    return Hinv
    
def coordinate_transformation(muk0s,sks):
    B = np.zeros((len(sks)*6,len(muk0s)*6))
    Pbar = np.zeros(len(sks)*6)
    for k in range(len(sks)):
        sig0 = np.linalg.inv(muk0s[k]) @ muk0s[k+1]
        Sig  = sig0[:3,:3]
        sig  = sig0[:3,3]
        Sk   = sks[k,:3,:3]
        sk   = sks[k,:3,3]
        
        Psi  = so3.rotmat2euler(Sk.T @ Sig)
        Hi   = Hinverse(Psi)
        Bkm = np.zeros((6,6))
        Bkp = np.zeros((6,6))
        Bkm[:3,:3] = -Hi @ Sig.T
        Bkm[3:,:3] = Sk.T @ so3.hat_map(sig)
        Bkm[3:,3:] = -Sk.T
        Bkp[:3,:3] = Hi
        Bkp[3:,3:] = Sk.T @ Sig
        
        B[6*k:6*(k+1),6*k:6*(k+1)]      = Bkm
        B[6*k:6*(k+1),6*(k+1):6*(k+2)]  = Bkp
        
        Pbar[k*6:k*6+3]   = Psi
        Pbar[k*6+3:k*6+6] = Sk.T @ (sig-sk)
    return B, Pbar

def coordinate_transformation_correction(muk0s,sks,Z_delta_ref):
    
    if len(Z_delta_ref.shape) < 2:
        Z_delta_ref = Z_delta_ref.reshape((len(Z_delta_ref)//6,6))
    
    B = np.zeros((len(sks)*6,len(muk0s)*6))
    Pbar = np.zeros(len(sks)*6)
    for k in range(len(sks)):
        sig0 = np.linalg.inv(muk0s[k]) @ muk0s[k+1]
        SIG  = sig0[:3,:3]
        sig  = sig0[:3,3]
        Sk   = sks[k,:3,:3]
        sk   = sks[k,:3,3]
        
        Psi  = so3.rotmat2euler(Sk.T @ SIG)
        Hi   = Hinverse(Psi)
        
        Z0k = so3.euler2rotmat(Z_delta_ref[k,:3])
        htheta0 = so3.hat_map(Z_delta_ref[k,:3])
        
        Bkm = np.zeros((6,6))
        Bkp = np.zeros((6,6))
        
        Bkm[:3,:3] = -Hi @ SIG.T
        Bkm[3:,:3] = Sk.T @ so3.hat_map(sig)
        Bkm[3:,3:] = -Sk.T @ Z0k.T
        
        Bkp[:3,:3] = Hi
        Bkp[3:,3:] = Sk.T @ Z0k.T @ SIG
        
        B[6*k:6*(k+1),6*k:6*(k+1)]      = Bkm
        B[6*k:6*(k+1),6*(k+1):6*(k+2)]  = Bkp
        
        Pbar[k*6:k*6+3]   = Psi
        Pbar[k*6+3:k*6+6] = Sk.T @ ( (Z0k.T + htheta0) @ sig-sk)
    return B, Pbar



### Free Energy

In [ ]:

def binding_model_free_energy(
    free_gs: np.ndarray,
    free_M: np.ndarray,    
    nuc_mu0: np.ndarray,
    nuc_K: np.ndarray,
    left_open: int,
    right_open: int,
    NUCLEOSOME_TRIADS: np.ndarray,
    use_correction: bool = True,
) -> np.ndarray:

    midstep_constraint_locations = get_midstep_locations(left_open, right_open)
    if len(midstep_constraint_locations) <= 1:
        n = len(free_M)
        F_pi = -0.5*n * np.log(2*np.pi)
        # matrix term
        logdet_sign, logdet = np.linalg.slogdet(free_M)
        F_mat = 0.5*logdet
        F = F_mat + F_pi  
        Fdict = {
            'F': F,
            'F_entropy' : F,
            'F_enthalpy': 0,
            'F_jacob'   : 0,
            'F_free'    : F
        }
        return Fdict
    
    # FOR NOW WE USE THE FIXED MIDSTEP TRIADS AS MU_0
    # Find midstep triads in fixed framework for comparison
    FIXED_midstep_triads = calculate_midstep_triads(
        midstep_constraint_locations,
        NUCLEOSOME_TRIADS
    )
    nuc_mu0 = FIXED_midstep_triads
    
    sks = midstep_groundstate_se3(free_gs,midstep_constraint_locations)
    
    # find composite transformation
    transform, replaced_ids = midstep_composition_transformation(
        free_gs,
        midstep_constraint_locations
    )
    
    # transform stiffness matrix
    inv_transform = np.linalg.inv(transform)
    M_transformed = inv_transform.T @ free_M @ inv_transform
    
    # rearrange stiffness matrix
    full_replaced_ids = list()
    for i in range(len(replaced_ids)):
        full_replaced_ids += [6*replaced_ids[i]+j for j in range(6)]
     
    P = send_to_back_permutation(len(free_M),full_replaced_ids)
    M_rearranged = P @ M_transformed @ P.T
    
    # select M and R submatrices
    N  = len(M_rearranged)
    NC = len(full_replaced_ids)
    NF = N-NC
    
    M_R = M_rearranged[:NF,:NF]
    M_M = M_rearranged[NF:,NF:]
    M_RM = M_rearranged[:NF,NF:]
    
    # Calculate M block marginal
    M_Mp = M_M - M_RM.T @ np.linalg.inv(M_R) @ M_RM
    M_Mp = 0.5*(M_Mp+M_Mp.T)
    

    ##############################################
    # Binding Model
    ##############################################
    
    nuc_K *= 1
    
    # Calculate Incidence Matrix
    B, Pbar = coordinate_transformation(nuc_mu0,sks)  
    Kcomb = nuc_K + B.T @ M_Mp @ B
    # calculate ground state
    alpha = -np.linalg.inv(Kcomb) @ B.T @ M_Mp @ Pbar
    
    B, Pbar = coordinate_transformation_correction(nuc_mu0,sks,alpha)
    Kcomb = nuc_K + B.T @ M_Mp @ B
    # calculate ground state
    alpha = -np.linalg.inv(Kcomb) @ B.T @ M_Mp @ Pbar
    
    
    Y_C = Pbar + B @ alpha
    C = 0.5* Pbar.T @ ( M_Mp - M_Mp @ B @ np.linalg.inv(Kcomb) @ B.T @ M_Mp ) @ Pbar
    # print(f'C = {C}')
    
    gamma = -np.linalg.inv(M_R) @ M_RM @ Y_C
    
    if use_correction:
    
        gs_transf_perm = np.concatenate((gamma,Y_C))
        gs_transf = P.T @ gs_transf_perm
        gs = inv_transform @ gs_transf

        gs = gs.reshape((len(gs)//6,6))
        # find composite transformation
        transform, replaced_ids = midstep_composition_transformation_correction_old(
            free_gs,
            midstep_constraint_locations,
            -gs
        )
        
        # transform stiffness matrix
        inv_transform = np.linalg.inv(transform)
        M_transformed = inv_transform.T @ free_M @ inv_transform
        
        # rearrange stiffness matrix
        full_replaced_ids = list()
        for i in range(len(replaced_ids)):
            full_replaced_ids += [6*replaced_ids[i]+j for j in range(6)]
        
        P = send_to_back_permutation(len(free_M),full_replaced_ids)
        M_rearranged = P @ M_transformed @ P.T
        
        # select M and R submatrices
        N  = len(M_rearranged)
        NC = len(full_replaced_ids)
        NF = N-NC
        
        M_R = M_rearranged[:NF,:NF]
        M_M = M_rearranged[NF:,NF:]
        M_RM = M_rearranged[:NF,NF:]
        
        # Calculate M block marginal
        M_Mp = M_M - M_RM.T @ np.linalg.inv(M_R) @ M_RM
        M_Mp = 0.5*(M_Mp+M_Mp.T)
        
        ##############################################
        # Binding Model
        ##############################################
        
        # Calculate Incidence Matrix
        B, Pbar = coordinate_transformation(nuc_mu0,sks)  
        Kcomb = nuc_K + B.T @ M_Mp @ B
        # calculate ground state
        alpha = -np.linalg.inv(Kcomb) @ B.T @ M_Mp @ Pbar
        
        B, Pbar = coordinate_transformation_correction(nuc_mu0,sks,alpha)
        Kcomb = nuc_K + B.T @ M_Mp @ B 
        
        
        
        # Kbare = nuc_K - B.T @ M_Mp @ B 
        # eigenvals, Q = np.linalg.eigh(Kbare)
        # print(eigenvals)
        
        # free_M = np.load('MDParams/free_midstep_Mm.npy')
        # Kbare = nuc_K - B.T @ free_M @ B 
        # eigenvals, Q = np.linalg.eigh(Kbare)
        # print(eigenvals)
        # sys.exit()
        
        # print(is_positive_definite(Kcomb))
        
        # calculate ground state
        alpha = -np.linalg.inv(Kcomb) @ B.T @ M_Mp @ Pbar
        
        Y_C = Pbar + B @ alpha
        gamma = -np.linalg.inv(M_R) @ M_RM @ Y_C
        
        C = 0.5* Pbar.T @ ( M_Mp - M_Mp @ B @ np.linalg.inv(Kcomb) @ B.T @ M_Mp ) @ Pbar
        # print(f'C = {C}')
        
    gs_transf_perm = np.concatenate((gamma,Y_C))
    gs_transf = P.T @ gs_transf_perm
    gs = inv_transform @ gs_transf
        
    F_enthalpy = C
    
    # Z entropy term
    n = len(Kcomb)
    logdet_sign, logdet_K = np.linalg.slogdet(Kcomb)
    F_piK = -0.5*n * np.log(2*np.pi)
    Z_entropy = 0.5*logdet_K + F_piK
    

    # Z entropy term
    n = len(M_R)
    logdet_sign, logdet_R = np.linalg.slogdet(M_R)
    F_piR = -0.5*n * np.log(2*np.pi)
    R_entropy = 0.5*logdet_R + F_piR
    
    # print(f'shape M_R = {M_R.shape}')
    
    
    F_jacob = np.log(np.linalg.det(transform))
    
    
    F_entropy = Z_entropy + R_entropy + F_jacob
    
    # free energy of unconstrained DNA
    ff_logdet_sign, ff_logdet = np.linalg.slogdet(free_M)
    ff_pi = -0.5*len(free_M) * np.log(2*np.pi)
    F_free = 0.5*ff_logdet + ff_pi


    # _, logdet1 = np.linalg.slogdet(M_R)
    # _, logdet2 = np.linalg.slogdet(M_Mp)
    
    # Mcomb = np.linalg.inv(B @ np.linalg.inv(Kcomb) @ B.T)
    # logdet_sign, logdet_K = np.linalg.slogdet(Mcomb)
    # F_piK = -0.5*len(Mcomb) * np.log(2*np.pi)
    # Mcomb_entropy = 0.5*logdet_K + F_piK
    
    # print(0.5*logdet1+0.5*logdet2+F_jacob -0.5*len(free_M) * np.log(2*np.pi) )
    # print(f'{Z_entropy=}')
    # print(f'{Mcomb_entropy=}')
    # print(f'{R_entropy=}')
    # print(f'{F_free=}')
    
    # prepare output
    Fdict = {
        'F': F_entropy + F_enthalpy - F_free,
        'F_entropy' : F_entropy - F_free,
        'F_enthalpy': F_enthalpy,
        'F_jacob'   : F_jacob,
        'F_free'    : F_free,
        'gs'        : gs
    }
    return Fdict

### Calculate K

In [261]:
fn_free_gs = 'MDParams/free_midstep_gs.npy'
fn_free_Mm = 'MDParams/free_midstep_Mm.npy'
fn_nuc_Kcomb = 'MDParams/nuc_midsteps_sym_Kcomb.npy'
fn_nuc_Kcomb = 'MDParams/nuc_midsteps_Kcomb.npy'

sks = np.load(fn_free_gs)
Mm  = np.load(fn_free_Mm)
Kcomb = np.load(fn_nuc_Kcomb) 


##################################################################
##################################################################
##################################################################

midstep_constraint_locations = get_midstep_locations(0, 0)

triadfn = 'methods/State/Nucleosome.state'
nuctriads = read_nucleosome_triads(triadfn)
mu0s = calculate_midstep_triads(
    midstep_constraint_locations,
    nuctriads
)

# Calculate Incidence Matrix
B, Pbar = coordinate_transformation(mu0s,sks)  
K = Kcomb - B.T @ Mm @ B


Kd = marginalize_diagonal_blocks(K)

print(is_positive_definite(K))
print(is_positive_definite(Kd))

Kd_reg = np.copy(Kd)

for i in range(len(K)//6):
    print('################################################################')
    print('################################################################')
    print(f'i={i}')
    Kdi = Kd[i*6:(i+1)*6,i*6:(i+1)*6]
    print(Kdi)
    print(is_positive_definite(Kdi))

    eigenvals, Q = np.linalg.eigh(Kdi)
    print(eigenvals)
    
    Kdi_reg = regularize_matrix(Kdi,epsilon=1)
    eigenvals, Q = np.linalg.eigh(Kdi_reg)
    print(eigenvals)
    
    print(Kdi_reg)
    
    
    Kd_reg[i*6:(i+1)*6,i*6:(i+1)*6] = Kdi_reg

sel = 18
Kiref = np.copy(Kd_reg[sel*6:(sel+1)*6,sel*6:(sel+1)*6])

for i in range(len(K)//6):
     Kd_reg[i*6:(i+1)*6,i*6:(i+1)*6] = Kiref
     


eigenvals, Q = np.linalg.eigh(Kcomb)
print(eigenvals[-1]/eigenvals[0])


False
False
################################################################
################################################################
i=0
[[ 302.06379 -182.3987  -152.37141 -574.26367 -159.57529  251.70659]
 [-182.3987   123.02837  163.91727  552.9589   154.90523 -312.02999]
 [-152.37141  163.91727  389.75658  509.85747  -47.49647 -185.57023]
 [-574.26367  552.9589   509.85747 1722.4849   596.08568 -801.1505 ]
 [-159.57529  154.90523  -47.49647  596.08568  755.21603 -329.44946]
 [ 251.70659 -312.02999 -185.57023 -801.1505  -329.44946  609.91379]]
False
[ -64.163     56.69995  147.12218  220.47741  657.87938 2884.44753]
[   1.        56.69995  147.12218  220.47741  657.87938 2884.44753]
[[ 302.08994 -181.15614 -152.37597 -574.54507 -159.49191  251.97705]
 [-181.15614  182.07278  163.70086  539.58714  158.8674  -299.17797]
 [-152.37597  163.70086  389.75738  509.90648  -47.511   -185.61734]
 [-574.54507  539.58714  509.90648 1725.51319  595.18837 -804.06109]
 [-159.49191  158.867

### Full diagonal

In [209]:
cov = np.linalg.inv(Kcomb)

dim = 6
avg_cov = np.zeros((dim,dim))

for i in range(len(cov)//6):
    avg_cov += cov[i*6:(i+1)*6,i*6:(i+1)*6]
    
avg_cov = avg_cov / (len(cov)//6)
Kavg = np.linalg.inv(avg_cov)

print(Kavg)

Kdiag = np.zeros(Kcomb.shape)
for i in range(len(K)//6):
     Kdiag[i*6:(i+1)*6,i*6:(i+1)*6] = Kavg


[[150.93032   0.       -0.00001  -3.66619  -0.        0.00001]
 [  0.      141.02478  31.12045   0.      -49.32606  -8.25888]
 [ -0.00001  31.12045 127.404     0.      -58.61702 -17.77438]
 [ -3.66619   0.        0.       74.89501  -0.00002   0.00002]
 [ -0.      -49.32606 -58.61702  -0.00002 233.55048 -19.1157 ]
 [  0.00001  -8.25888 -17.77438   0.00002 -19.1157  195.63549]]


### Play with matrix

In [185]:
diags = np.ones(len(Kcomb))*5
diags[0::6] = 50
diags[1::6] = 50
diags[2::6] = 50
K = np.diag(diags)


# Calculate Incidence Matrix
B, Pbar = coordinate_transformation(mu0s,sks)    
Kcomb_ = K + B.T @ Mm @ B
# calculate ground state
alpha = -np.linalg.inv(Kcomb_) @ B.T @ Mm @ Pbar

B, Pbar = coordinate_transformation_correction(mu0s,sks,alpha)
Kcomb_ = K + B.T @ Mm @ B

print(is_positive_definite(Kcomb_))

# calculate ground state
alpha = -np.linalg.inv(Kcomb_) @ B.T @ Mm @ Pbar

mud = alpha.reshape((len(alpha)//6,6))
# print(mud[:,:3]*180/np.pi)
print(mud[:,3:])


True
[[-0.06738  0.19339  0.05893]
 [ 0.0985  -0.02616  0.04455]
 [-0.08511  0.02384  0.02407]
 [-0.04194  0.04269  0.0383 ]
 [ 0.06202 -0.05538  0.06464]
 [-0.04313  0.02589  0.06942]
 [-0.06055 -0.00982  0.03252]
 [ 0.0956  -0.07243  0.01115]
 [-0.04872  0.2041  -0.04549]
 [ 0.06592 -0.11149 -0.00703]
 [ 0.0502  -0.01193 -0.03155]
 [-0.18143 -0.07213  0.07865]
 [ 0.0302   0.11895  0.02009]
 [ 0.04306 -0.13753  0.01296]
 [ 0.02711  0.10196 -0.03919]
 [-0.01382 -0.09859 -0.03941]
 [-0.17888  0.06217 -0.09773]
 [ 0.06073 -0.00953  0.0229 ]
 [ 0.08172  0.0934   0.01031]
 [-0.03428 -0.17088  0.02505]
 [ 0.08049  0.09241 -0.0438 ]
 [-0.04938 -0.02904 -0.06394]
 [-0.05304 -0.02658 -0.13741]
 [ 0.06064  0.09734 -0.12419]
 [-0.01712 -0.04511 -0.0151 ]
 [-0.0872  -0.0692  -0.00207]
 [ 0.1461   0.05139 -0.02202]
 [-0.04609 -0.2324  -0.04229]]


In [7]:

genstiff = GenStiffness(method='hybrid')
triadfn = 'methods/State/Nucleosome.state'
nuctriads = read_nucleosome_triads(triadfn)

BM_Kcomb_fn = 'MDParams/Calibrated/reg_Kcomb.npy'
BM_K_fn     = 'MDParams/Calibrated/reg_K.npy'
BM_mu0_fn   = 'MDParams/Calibrated/reg_mu0.npy'


nuc_Kcomb = np.load(BM_Kcomb_fn)
nuc_K     = np.load(BM_K_fn)
nuc_mu0   = np.load(BM_mu0_fn)


nuc_Kcomb = np.load('MDParams/nuc_midsteps_sym_Kcomb.npy')
nuc_Kcomb = np.load('MDParams/nuc_midsteps_Kcomb.npy')




left_open  = 0
right_open = 0     
  
seq = "ATCAATATCCACCTGCAGATACTACCAAAAGTGTATTTGGAAACTGCTCCATCAAAAGGCATGTTCAGCTGGAATCCAGCTGAACATGCCTTTTGATGGAGCAGTTTCCAAATACACTTTTGGTAGTATCTGCAGGTGGATATTGAT"  
# seq = "CTGGAGAATCCCGGTGCCGAGGCCGCTCAATTGGTCGTAGACAGCTCTAGCACCGCTTAAACGCACGTACGCGCTGTCCCCCGCGTTTTAACCGCCAAGGGGATTACTCCCTAGTCTCCAGGCACGTGTCAGATATATACATCCTGT"

seq = 'ATTTGGCCTTAAAAAAACTTCCCCCTTCGCTATACAAGAGATTCATCGGAAAGATCAGTGGAGCGAACTGCCCTACATCATCCTAATGAGATTCGGTGCTGTTAATTGGGTACCAGACTTCCACGCGAAAAAATCGCGGGGGCACGA'
seq = 'ATTTGGCCTTAAAAAAACTTCCCCCTTCGCTATACAAGAGATTCATCGGAAAGATCAGTGGAGCGAACTGCCCTACATCATCCTAATGAGATTCGGTGCTGTTAATTGGGTACCAGACTTCCACGCGAAAAAATCGCGGGGGCACGA'

seq = ''.join(['ATCG'[np.random.randint(4)] for i in range(147)])

free_M,free_gs = genstiff.gen_params(seq,use_group=True)

        
# nucleosome_free_energy(groundstate,stiffmat,left_open,right_open,nuctriads,K)

# K = marginalize_diagonal_blocks(nuc_Kcomb)
# K = np.copy(Kd_reg)
# K = np.copy(Kd1)

diags = np.ones(len(nuc_Kcomb))*1
diags[0::6] = 1
diags[1::6] = 1
diags[2::6] = 1
K = np.diag(diags)

# K = np.eye(len(nuc_Kcomb))*20

# K = marginalize_diagonal_blocks(K)

# K = np.copy(Kdiag)
# K = np.copy(Kcomb)

F = binding_model_free_energy(
    free_gs,
    free_M,    
    nuc_mu0,
    # nuc_Kcomb,
    K,
    left_open,
    right_open,
    nuctriads,
    use_correction = True
)
print(F)

plot_matrix(K)

# Calculate Incidence Matrix
B, Pbar = coordinate_transformation(mu0s,sks)    
Kcomb_ = K + B.T @ Mm @ B
# calculate ground state
alpha = -np.linalg.inv(Kcomb_) @ B.T @ Mm @ Pbar

B, Pbar = coordinate_transformation_correction(mu0s,sks,alpha)
Kcomb_ = K + B.T @ Mm @ B

plot_matrix(Mm)
plot_matrix(B.T @ Mm @ B)

_, logdet = np.linalg.slogdet(Mm)
print(0.5*logdet)
_, logdet = np.linalg.slogdet(K)
print(0.5*logdet)


[-1339.73113 -1230.2769  -1115.67294 -1062.26326  -989.76945  -971.07546  -926.63552  -871.16845  -856.19742  -793.05222  -763.88916  -729.90126  -655.32821  -643.95092  -377.7294   -359.52514  -346.14901  -325.83572  -315.50457  -293.36752
  -288.16968  -286.33584  -283.25565  -273.29724  -270.72035  -263.21766  -257.9954   -253.40412  -250.63693  -246.73305  -234.92458  -233.86349  -231.09249  -226.42317  -224.97073  -224.53038  -220.9276   -217.5271   -216.50802  -215.38328
  -212.71307  -212.06754  -210.5134   -207.60455  -206.60855  -206.58966  -204.39199  -202.99003  -201.61806  -198.75201  -195.95793  -194.24055  -191.4642   -188.88366  -187.40412  -183.73348  -180.78785  -179.46328  -175.19049  -172.36419
  -171.05203  -164.72865  -158.97366  -157.78386  -153.16121  -151.71806  -148.48327  -145.47823  -144.40386  -140.49264  -138.27343  -136.97998  -135.05501  -133.49337  -127.77033  -127.12949  -123.66662  -120.86559  -118.6972   -116.77231
  -114.51846  -114.22043  -111.93426

SystemExit: 

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:

for i in range(28):
    seq = ''.join(['ATCG'[np.random.randint(4)] for i in range(147)])

    free_M,free_gs = genstiff.gen_params(seq,use_group=True)
    diags = np.ones(len(nuc_Kcomb))*10
    diags[0::6] = 1
    diags[1::6] = 1
    diags[2::6] = 1
    K = np.diag(diags)
    
    # K = np.copy(Kcomb)

    F = binding_model_free_energy(
        free_gs,
        free_M,    
        nuc_mu0,
        # nuc_Kcomb,
        K,
        left_open,
        right_open,
        nuctriads,
        use_correction = True
    )
    print(f'FE: {F["F"]}, S: {F["F_entropy"]}, H: {F["F_enthalpy"]}')



FE: 101.80436525632945, S: 51.0311124672221, H: 50.77325278910728
FE: 104.67145564096336, S: 50.86875724770994, H: 53.80269839325348
FE: 103.17695241775277, S: 50.83584599681694, H: 52.3411064209358
FE: 101.40551992061432, S: 50.901563796477376, H: 50.50395612413698
FE: 104.26343055825714, S: 50.684106297163, H: 53.57932426109417
FE: 110.71750945582698, S: 50.66746188180923, H: 60.050047574017825
FE: 102.62559275535182, S: 50.9894242046023, H: 51.636168550749446
FE: 107.22198394497832, S: 50.91532252078332, H: 56.30666142419505
FE: 102.31475759309888, S: 50.759543258502845, H: 51.55521433459608
FE: 104.12057417490155, S: 50.93350933333454, H: 53.18706484156701
FE: 102.45916481993208, S: 50.82390075770422, H: 51.63526406222781
FE: 102.79390429961904, S: 50.72135613353839, H: 52.07254816608073
FE: 104.50194535726087, S: 50.844003188659826, H: 53.65794216860113
FE: 104.08541327163175, S: 50.99114010208541, H: 53.09427316954643
FE: 107.19234916989922, S: 50.68540842329071, H: 56.5069407466